# Doppler Solve

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry

In [ ]:
def generate(
    nc=1,
    show=True,
    flux_err=1e-6,
    ydeg=15,
    nt=32,
    inc=75,
    veq=50000,
    smoothing=0.075,
    **kwargs
):

    # Instantiate
    map = starry.DopplerMap(ydeg=ydeg, nc=nc, veq=veq, inc=inc, nt=nt)

    # Component surface images
    if nc == 1:
        images = ["spot"]
    elif nc == 2:
        images = ["star", "spot"]
    else:
        raise NotImplementedError("")

    # Component spectra
    if nc == 1:
        mu = np.array([643.0])
    elif nc == 2:
        mu = np.array([643.1, 642.9])
    else:
        raise NotImplementedError("")
    sig = 0.025
    dw = map.wav0.reshape(1, -1) - mu.reshape(-1, 1)
    spectra = (1.0 - np.exp(-0.5 * dw ** 2 / sig ** 2))[:nc]

    # Load the component maps
    map.load(images=images, spectra=spectra, smoothing=smoothing)

    # Show
    if show:
        map.show_components(show_spectra=True)

    # Generate unnormalized data
    flux0 = map.flux(normalize=False)
    flux0 += flux_err * np.random.randn(*flux0.shape)

    # Generate normalized data
    flux = map.flux(normalize=True)
    flux += flux_err * np.random.randn(*flux.shape)

    return map, flux0, flux

## Solve for the map

### One component

#### Generate the data

In [ ]:
settings = dict(flux_err=1e-6, ydeg=15, nt=16, inc=60, veq=50000, smoothing=0.075)
map, flux0, flux = generate(nc=1, **settings)

In [ ]:
fig, ax = plt.subplots(1, figsize=(5, 5))
ax.plot(map.wav, flux.T + np.linspace(0, 1.5, map.nt).reshape(1, -1), color="k", lw=1)
ax.set_xlabel("wavelength [nm]")
ax.set_ylabel("intensity + offset");

#### Unnormalized solve

In [ ]:
map.solve(flux0, normalized=False, fix_spectrum=True, flux_err=settings["flux_err"])
map.show_components(show_spectra=False)

#### Normalized solve w/ tempering

With the default prior:

In [ ]:
map.solve(flux, normalized=True, fix_spectrum=True, flux_err=settings["flux_err"])
map.show_components(show_spectra=False, vmin=0, vmax=0.52)

With a stricter prior to suppress features at the south pole:

In [ ]:
spatial_cov = np.ones(map.Ny)
spatial_cov[1:] = 1e-7
map.solve(
    flux,
    normalized=True,
    fix_spectrum=True,
    flux_err=settings["flux_err"],
    spatial_cov=spatial_cov,
)
map.show_components(show_spectra=False)

### Two components

#### Generate the data

In [ ]:
settings = dict(flux_err=1e-6, ydeg=15, nt=16, inc=60, veq=50000, smoothing=0.075)
map, flux0, flux = generate(nc=2, **settings)

In [ ]:
fig, ax = plt.subplots(1, figsize=(5, 5))
ax.plot(map.wav, flux.T + np.linspace(0, 1.5, map.nt).reshape(1, -1), color="k", lw=1)
ax.set_xlabel("wavelength [nm]")
ax.set_ylabel("intensity + offset");

#### Unnormalized solve

In [ ]:
map.solve(flux0, normalized=False, fix_spectrum=True, flux_err=settings["flux_err"])
map.show_components(show_spectra=False)

#### Normalized solve w/ tempering

In [ ]:
map.solve(flux, normalized=True, fix_spectrum=True, flux_err=settings["flux_err"])
map.show_components(show_spectra=False)

## Solve for everything

### One component

#### Generate the data

In [ ]:
settings = dict(flux_err=1e-6, ydeg=15, nt=16, inc=60, veq=50000, smoothing=0.075)
map, flux0, flux = generate(nc=1, **settings)

In [ ]:
fig, ax = plt.subplots(1, figsize=(5, 5))
ax.plot(map.wav, flux.T + np.linspace(0, 1.5, map.nt).reshape(1, -1), color="k", lw=1)
ax.set_xlabel("wavelength [nm]")
ax.set_ylabel("intensity + offset");